In [ ]:
import wave
import numpy as np
from scipy.io import wavfile
from scipy.signal import find_peaks,butter, sosfreqz, sosfilt,spectrogram
import sounddevice as sd
import scipy.io.wavfile as wav
from pydub import AudioSegment
from pydub.playback import play


import matplotlib.pyplot as plt

from IPython.display import Audio

In [ ]:
!pip3 uninstall sounddevice


In [ ]:

def load_audio(file_path):
    rate, data = wav.read(file_path)
    return rate, data

def plot_spectrum(rate, data, file_name, ax):
    f, t, Sxx = spectrogram(data, fs=rate, nperseg=1024)
    im = ax.pcolormesh(t, f, 10 * np.log10(Sxx))
    ax.set_title(f'Frequency Spectrum - {file_name}')
    ax.set_ylabel('Frequency [Hz]')
    ax.set_xlabel('Time [sec]')
    return im

def detect_ultrasonic_signal(audio_file):
      # Perform FFT to analyze frequency content and locate peaks
    sample_rate, data = wavfile.read(audio_file)
    spectrum = np.fft.fft(data)
    frequencies = np.fft.fftfreq(len(spectrum), d=1/sample_rate)
    peaks, _ = find_peaks(np.abs(spectrum), height=1000)  # Adjust height threshold as needed
    # Filter out low-frequency peaks 
    ultrasonic_peaks = [freq for freq in frequencies[peaks] if freq > 22200]
    return ultrasonic_peaks[0] if ultrasonic_peaks else None


def demodulate_amplitude(rate, data, ultrasonic_frequency):
    t = np.arange(len(data)) / rate
    demodulated_signal = data * np.cos(2 * np.pi * ultrasonic_frequency * t)
    return demodulated_signal

def play_secret_code(secret_code, audible_frequency, rate, output_file="output.wav"):
    secret_code = secret_code.astype(np.int16)
    audio_data = secret_code * np.cos(2 * np.pi * audible_frequency * np.arange(len(secret_code)) / rate)
    audio_data_normalized = (audio_data / np.max(np.abs(audio_data))) * 32767
    # Save the audio to a .wav file
    wavfile.write(output_file, rate, audio_data_normalized.astype(np.int16))
    # Play the normalized audio using sounddevice
    sd.play(audio_data_normalized, rate)
    print("Secret code audio playing now... Listen well")
    sd.wait()

file_paths = ['Ex3_sound2.wav', 'Ex3_sound3.wav', 'Ex3_sound4.wav']
audible_frequency = 20 

fig, axs = plt.subplots(len(file_paths), 1, figsize=(10, 6 * len(file_paths)))

for i, file_path in enumerate(file_paths):
    rate, data = load_audio(file_path)
    im = plot_spectrum(rate, data, file_path, axs[i])
    
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='Power/Frequency [dB/Hz]')
plt.show()

# Detect ultrasonic signals and play the secret code
for file_path in file_paths:
    rate, data = load_audio(file_path)

    ultrasonic_frequency = detect_ultrasonic_signal(file_path)
    if ultrasonic_frequency:
        demodulated_signal = demodulate_amplitude(rate, data, ultrasonic_frequency)
        play_secret_code(demodulated_signal, audible_frequency, rate, output_file="output.wav")

        print(f"Ultrasonic-frequency found for {file_path} !!!")
       
    else:
        print(f"No ultrasonic-frequency found for {file_path}")
